# Fine-Tuning Gemma 3B with LoRA on a Custom Dataset (Colab)

This notebook shows how to fine-tune Google's Gemma# 1B model using the LoRA method on your own academic dataset in JSON format, using Kaggle API to download the model — all within Google Colab!

---

## Step 1: Install Required Libraries


In [ ]:
!pip install -q -U keras-hub
!pip install  -q -U keras

## Step 2: Setup Kaggle API to Download Gemma 3B

- Go to [Kaggle Account Settings](https://www.kaggle.com/settings)
- Click “Create New API Token”
- Upload `kaggle.json` in Colab or set it via Colab Secrets


In [ ]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

In [ ]:
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

## Step 3: Download Gemma 3B Model from Kaggle


In [ ]:
import keras
import keras_hub

In [ ]:
gemma_lm = keras_hub.models.Gemma3CausalLM.from_preset("gemma3_instruct_1b")
gemma_lm.summary()

Preprocessor: "gemma3_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma3_tokenizer (Gemma3Tokenizer)                            │                      Vocab size: 262,144 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma3_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma3_backbone               │ (None, None, 1152)        │     999,885,952 │ padding_mask[0][0],        │
│ (Gemma3Backbone)              │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 262144)      │     301,989,888 │ gemma3_backbone[0][0]      │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 999,885,952 (3.72 GB)

 Trainable params: 999,885,952 (3.72 GB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
template = "Instruction:\n{instruction}\n\nResponse:\n{response}"

prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
sampler = keras_hub.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
What should I do on a trip to Europe?

Response:
Europe offers a huge and diverse range of experiences! To help you plan the perfect trip, here's a breakdown of suggestions based on different interests and budgets:

**1. Popular Destinations (Good for First-Timers):**

*   **Paris:** Iconic landmarks like the Eiffel Tower, Louvre Museum, and Notre Dame Cathedral. Romantic atmosphere, world-class cuisine.
*   **Rome:** Ancient history, stunning architecture (Colosseum, Roman Forum), delicious pasta and pizza.
*   **London:** Royal history, vibrant theater scene, museums galore (British Museum, National Gallery).
*   **Barcelona:** Unique architecture (Gaudí's Sagrada Familia), beaches, lively nightlife.
*   **Amsterdam:** Canals, museums (Rijksmuseum, Van Gogh Museum), charming architecture, cycling culture.

**2. Budget-Friendly Options:**

*   **Portugal:** Affordable cost of living, beautiful coastline, delicious seafood, charming towns.
*   **Czech Republic:** Historic 

In [ ]:
prompt = template.format(
    instruction="Explain the names of coffee",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
Explain the names of coffee

Response:
Here's an explanation of some common coffee names:

*   **Arabica:** This is the most popular and prized coffee bean variety. It’s known for its smoother, sweeter flavor and lower acidity.

*   **Robusta:** This is a stronger, more bitter coffee bean with higher caffeine content. It is often used in espresso blends due to its resilience.

*   **Liberica:** A unique and often polarizing coffee bean with a smoky, woody, and floral aroma. It's less common than Arabica or Robusta.

*   **Excelsa:** A rarer variety that's often used for its tart, fruity flavor and unique aromatic profile.

*   **Sensual:** A light-bodied, fruity coffee with a delicate aroma. It's often described as having notes of berry and citrus.

*   **Dark Roast:** This describes coffee beans that have been subjected to a long roasting process, resulting in a dark color and bold flavor.

*   **Light Roast:** This coffee beans have undergone a minimal roasting, retainin

## Step 4: Upload & Load Your Custom Dataset (JSON)
- The dataset format should be like:




In [ ]:
import json

coffee_data = [
{"instruction": "Explain the names of coffee?", "context": "", "response": "Madras Filter coffee, Malabar coffee,Spiced Cardamom Coffee, Coorg coffee", "category": "coffee"},
{"instruction": "What is Madras Filter coffee?", "context": "", "response": "Madras Filter coffee is a traditional South Indian coffee made with a metal filter, served hot and frothy in a steel tumbler.", "category": "coffee"},
{"instruction": "Tell me about Monsoon Malabar coffee.", "context": "", "response": "Monsoon Malabar is a unique Indian coffee from Kerala aged in monsoon winds, resulting in a mellow, earthy flavor.", "category": "coffee"},
{"instruction": "What is Coorg coffee known for?", "context": "", "response": "Coorg coffee, grown in Karnataka, is known for its balanced body, rich aroma, and chocolatey notes.", "category": "coffee"},
{"instruction": "How is Chikmagalur coffee different?", "context": "", "response": "Chikmagalur coffee is famous for its Arabica beans and smooth taste, often used for cold brews and light roasts.", "category": "coffee"},
{"instruction": "What is special about Spiced Cardamom Coffee?", "context": "", "response": "Spiced Cardamom Coffee combines traditional Indian spices with filter coffee, offering a fragrant and flavorful experience.", "category": "coffee"},
{"instruction": "Describe Jaggery Coffee.", "context": "", "response": "Jaggery Coffee is sweetened with unrefined jaggery instead of sugar, giving it a deep, caramel-like sweetness and rustic charm.", "category": "coffee"},
{"instruction": "What is Saffron Kahwa Latte?", "context": "", "response": "Saffron Kahwa Latte is a fusion of Kashmiri kahwa and milk-based coffee, flavored with saffron, cinnamon, and almonds.", "category": "coffee"},
{"instruction": "What is Turmeric Coffee?", "context": "", "response": "Turmeric Coffee is a health-focused Indian coffee made by adding turmeric to espresso, known for its anti-inflammatory properties.", "category": "coffee"},
{"instruction": "What is Ginger Filter Coffee?", "context": "", "response": "Ginger Filter Coffee is a spiced South Indian coffee infused with fresh ginger for an aromatic and energizing flavor.", "category": "coffee"},
{"instruction": "What is Rose Masala Cappuccino?", "context": "", "response": "Rose Masala Cappuccino blends floral rose essence with Indian masala spices, creating a fragrant and warming coffee.", "category": "coffee"},

]

with open("indian_coffee.jsonl", "w") as file:
    for entry in coffee_data:
        file.write(json.dumps(entry) + "\n")


In [ ]:
import json

prompts = []
responses = []
line_count = 0

with open("indian_coffee.jsonl") as file:
    for line in file:
        if line_count >= 10:
            break  # Limit the training examples, to reduce execution time.

        examples = json.loads(line)
        # Filter out examples with context, to keep it simple.
        if examples["context"]:
            continue
        # Format data into prompts and response lists.
        prompts.append(examples["instruction"])
        responses.append(examples["response"])

        line_count += 1

data = {
    "prompts": prompts,
    "responses": responses
}



## Step 5: Apply LoRA Configuration and Training Settings
# Enable LoRA for the model and set the LoRA rank to 4.



In [ ]:
gemma_lm.backbone.enable_lora(rank=4)

In [ ]:
gemma_lm.summary()

Preprocessor: "gemma3_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma3_tokenizer (Gemma3Tokenizer)                            │                      Vocab size: 262,144 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma3_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma3_backbone               │ (None, None, 1152)        │   1,000,538,240 │ padding_mask[0][0],        │
│ (Gemma3Backbone)              │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 262144)      │     301,989,888 │ gemma3_backbone[0][0]      │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 1,000,538,240 (3.73 GB)

 Trainable params: 652,288 (2.49 MB)

 Non-trainable params: 999,885,952 (3.72 GB)

In [ ]:
# Limit the input sequence length to 256 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 256
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Step 6: Train the Model

In [ ]:
gemma_lm.fit(data, epochs=1, batch_size=1)

10/10 ━━━━━━━━━━━━━━━━━━━━ 534s 43s/step - loss: 0.3903 - sparse_categorical_accuracy: 0.4553


# Step 7: Test The Model

In [ ]:
template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
prompt = template.format(
    instruction="Explain the names of coffee",
    response="",
)
sampler = keras_hub.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))